### doc_entity
ESTABELECE RELAÇÕES ENTRE DOCUMENTOS E ENTIDADES NA BASE DE DADOS NO MYSQL. TRABALHAMOS COM O ACERVO **ANTONIO AZEREDO DA SILVEIRA, MINISTÉRIO DAS RELAÇÕES EXTERIORES**.  

AS ENTIDADES A SEREM TRABALHADAS SÃO:
* PAÍSES
* PESSOAS

In [31]:
import os
import sys
import numpy as np
import pandas as pd
import re
from IPython.display import clear_output
import csv
import getpass
import pymysql
import string
import pickle
import random
import unicodedata
import collections
import nltk

In [2]:
sql_user='marcelobribeiro'

In [15]:
path_outputs = '../../text-learning-tools/outputs/'

In [4]:
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

### Corrections on names that had errors:
* Castro Alves is actually Dario (Moreira) Castro Alves.
* U Nu was a Burmese politician that was incorrectly mined, because all the parameters made weren't enough to handle this type of name.
* John J. Kennedy is ambiguous with John F. Kennedy.
* Vieira Souto was mined incorrectly, even using 'palavras'. The name is always avenue, but was often mined as person.
* renamed persons which only had surnames: 
    - debernardi
    - allara
    - sauvagnargues

OBS: to search neighbor tokens (of persons) using nltk, see function_drafts file.

** correct names on database ** 

In [6]:
pass_mysql = getpass.getpass()
conn = pymysql.connect(host='localhost', 
                       user=sql_user,
                       passwd=pass_mysql,
                       db='CPDOC_AS',
                       use_unicode=True, 
                       charset="utf8")
cur = conn.cursor()

cur.execute("UPDATE person_doc SET person_id='500272' WHERE person_id='500222'") # Dario Castro Alves
cur.execute("DELETE FROM person_doc WHERE person_id='109955'") # U Nu 
cur.execute("DELETE FROM person_doc WHERE person_id='500988'") # Vieira Souto
cur.execute("DELETE FROM person_doc WHERE person_id='107026'") # John J. Kennedy

# corrected on doc_entities_person_names_list
cur.execute("UPDATE persons SET person_name='enzo debernardi' WHERE id='500275'") # Debernardi
cur.execute("UPDATE persons SET person_name='gualter oscar allara' WHERE id='500056'") # Allara
cur.execute("UPDATE persons SET person_name='jean sauvagnargues' WHERE id='500942'") # Sauvagnargues    

········


1

## correct duplicates on frus, kissinger and clinton DBs

### Captures list of  names at History-Lab database
Tables are:
* declassification_frus - already used
* declassification_cables - no persons table
* declassification_ddrs - no persons table
* declassification_kissinger
* declassification_clinton - ok

### function of name extraction

In [8]:
def extract_names(data, names_list, names_ids_list):
    for row in data:
        if row is None: break
        var_names = []
        name_id = row[0]
        fullname =  str(row[1])
        fullname = str.lower(fullname)
        #print('1', fullname)
        fullname = re.sub('(.*?)\;.*', r'\1', fullname) #take out variants
        fullname = re.sub('^b[\"\']', r'', fullname) #take out quotation mark
        fullname = re.sub('[\"\']$', r'', fullname) #take out quotation mark
        fullname = re.sub('[^ ]\(.*\) ', r'', fullname) #take out nicknames in parenthesis
        fullname = re.sub(' \(.*\)[^ ]', r'', fullname) #take out nicknames in parenthesis
        fullname = re.sub('(.*), (.*)', r'\2 \1', fullname) #name format, from english to portuguese
        fullname = re.sub('(.*)\(.*\)\s?(.*)', r'\1\2', fullname)
        #print('2', fullname)
        fullname = re.sub('- ', '-', fullname)
        fullname_strip_accent = strip_accents(fullname)
        fullname_strip_abrev = re.sub('(\w+\. ){0,4}', r'', fullname)
        fullname_strip_both = strip_accents(fullname_strip_abrev)
        var_names.append(fullname)
        if fullname_strip_accent != fullname: var_names.append(fullname_strip_accent)
        if fullname_strip_abrev  != fullname: var_names.append(fullname_strip_abrev)
        if fullname_strip_both   != fullname_strip_abrev and fullname_strip_both != fullname_strip_accent: 
            var_names.append(fullname_strip_both)
        names_list[fullname] = var_names
        name_id_pair = [name_id, fullname]
        names_ids_list.append(name_id_pair)

In [59]:
data

()

### names extraction from databases
Databases are: FRUS, kissinger, clinton.

In [9]:
frus_names_list = {}
frus_names_ids_list = []

pass_mysql = getpass.getpass()
conn = pymysql.connect(host='history-lab.org', 
                       user='de_reader',
                       passwd=pass_mysql,
                       db='declassification_frus',
                       use_unicode=True, 
                       charset="utf8")
cur = conn.cursor()
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

cur.execute("SELECT id, name FROM persons")
data = cur.fetchall()

extract_names(data, frus_names_list, frus_names_ids_list)

········


In [10]:
clinton_names_list = {}
clinton_names_ids_list = []

pass_mysql = getpass.getpass()
conn = pymysql.connect(host='history-lab.org', 
                       user='de_reader',
                       passwd=pass_mysql,
                       db='declassification_clinton',
                       use_unicode=True, 
                       charset="utf8")
cur = conn.cursor()
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

cur.execute("SELECT id, name FROM persons")
data = cur.fetchall()

extract_names(data, clinton_names_list, clinton_names_ids_list)

········


In [11]:
kissinger_names_list = {}
kissinger_names_ids_list = []

pass_mysql = getpass.getpass()
conn = pymysql.connect(host='history-lab.org', 
                       user='de_reader',
                       passwd=pass_mysql,
                       db='declassification_kissinger',
                       use_unicode=True, 
                       charset="utf8")
cur = conn.cursor()
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

cur.execute("SELECT id, name FROM persons")
data = cur.fetchall()

extract_names(data, kissinger_names_list, kissinger_names_ids_list)

········


### collect duplicated names

In [18]:
cpdoc_names_check_file = path_outputs+'corpus_entities_persons_list_cpdoc.pkl'
cpdoc_names_check = pickle.load(open(cpdoc_names_check_file, 'rb'))

In [20]:
id_names_list = []
for name, values in cpdoc_names_check.items():
    for alt_name in values:
        for frus_name, frus_values in frus_names_list.items():
            for frus_alt_name in frus_values:
                if alt_name == frus_alt_name:
                    for id_name_tuple in frus_names_ids_list:
                        if frus_name == id_name_tuple[1]:
                            name_id = id_name_tuple[0]
                            cpdoc_id_name = [name_id, name, 'frus and cpdoc']
                            id_names_list.append(cpdoc_id_name)
        for kis_name, kis_values in kissinger_names_list.items():
            for kis_alt_name in kis_values:
                if alt_name == kis_alt_name:
                    for id_name_tuple in kissinger_names_ids_list:
                        if kis_name == id_name_tuple[1]:
                            name_id = id_name_tuple[0]
                            cpdoc_id_name = [name_id, name, 'kissinger and cpdoc']
                            id_names_list.append(cpdoc_id_name)
        for cli_name, cli_values in clinton_names_list.items():
            for cli_alt_name in cli_values:
                if alt_name == cli_alt_name:
                    #print(alt_name)
                    for id_name_tuple in clinton_names_ids_list:
                        if cli_name == id_name_tuple[1]:
                            #print('cli', cli_name)
                            name_id = id_name_tuple[0]
                            cpdoc_id_name = [name_id, name, 'clinton and cpdoc']
                            id_names_list.append(cpdoc_id_name)           

### check for duplicated ids

** remove completely duplicated lines **

In [21]:
id_names_list_rmdupl = []
for value in id_names_list:
    if value not in id_names_list_rmdupl:
        id_names_list_rmdupl.append(value)
id_names_list = id_names_list_rmdupl

** remove lines with duplicated ids **

In [22]:
id_duplicates_check = []
ids = []
for i in id_names_list:
    id_check = int(i[0])
    ids.append(id_check)
id_duplicates_check = [item for item, count in collections.Counter(ids).items() if count > 1]

In [23]:
id_names_list.remove(['107026', 'john kennedy', 'frus and cpdoc'])
for i in id_names_list:
    if int(i[0]) in id_duplicates_check and i[2] != 'frus and cpdoc': 
        id_names_list.remove(i)

** remove duplicated names **

In [46]:
id_names_list[:4]

[[7162, 'mário soares', 'clinton and cpdoc'],
 ['114246', 'vernon walters', 'frus and cpdoc'],
 [8079, 'vernon walters', 'clinton and cpdoc'],
 ['116279', 'frederick wills', 'frus and cpdoc']]

In [53]:
for line in id_names_list:
    prev_line = id_names_list[id_names_list.index(line)-1]
    if line[1] == prev_line[1]:
        id_names_list.remove(line)

### analyse database

In [24]:
duplicate_names = []
count_names = []
pass_mysql = getpass.getpass()
conn = pymysql.connect(host='localhost', 
                       user=sql_user,
                       passwd=pass_mysql,
                       db='CPDOC_AS',
                       use_unicode=True, 
                       charset="utf8")
cur = conn.cursor()

for i in id_names_list:
    p_id = i[0]
    p_name = i[1]
    p_source = i[2]
    if p_source == 'frus and cpdoc':
        query = "SELECT * FROM persons WHERE (id =%s OR person_name=%s)"
        cur.execute(query, (p_id, p_name))
        duplicate_name = cur.fetchall()
        duplicate_names.append(duplicate_name)
for line in duplicate_names:
    p_id1 = line[0][0]
    p_id2 = line[1][0]
    query = """
    SELECT person_id, COUNT(person_id) AS sum_id FROM person_doc 
    WHERE person_id = %s
    GROUP BY person_id
    """
    cur.execute(query, (p_id1))
    eval_name1 = cur.fetchall()
    cur.execute(query, (p_id2))
    eval_name2 = cur.fetchall()
    eval_names = [eval_name1, eval_name2]
    print('1',line)
    print('2',eval_names)
    count_names.append(eval_names)

········


IndexError: tuple index out of range

In [25]:
duplicate_names

[(('114246', 'vernon walters', 'frus and cpdoc', None, None, None),),
 (('116279', 'frederick wills', 'frus and cpdoc', None, None, None),),
 (('104230', 'raul fernandes', 'frus and cpdoc', None, None, None),),
 (('115232', 'juan maria bordaberry', 'frus and cpdoc', None, None, None),),
 (('111914', 'hernan santa cruz', 'frus and cpdoc', None, None, None),),
 (('101368', 'eugene black', 'frus and cpdoc', None, None, None),),
 (('106300', 'samuel huntington', 'frus and cpdoc', None, None, None),),
 (('109882', 'richard nixon', 'frus and cpdoc', None, None, None),),
 (('111954', 'eisaku sato', 'frus and cpdoc', None, None, None),),
 (('102333', 'nicolae ceausescu', 'frus and cpdoc', None, None, None),),
 (('115847', 'aparício mendez', 'frus and cpdoc', None, None, None),),
 (('102566', 'winston churchill', 'frus and cpdoc', None, None, None),),
 (('107024', 'edward kennedy', 'frus and cpdoc', None, None, None),),
 (('104698', 'carlos garcia', 'frus and cpdoc', None, None, None),),
 (('10

### alter ids

In [56]:
pass_mysql = getpass.getpass()
conn = pymysql.connect(host='localhost', 
                       user=sql_user,
                       passwd=pass_mysql,
                       db='CPDOC_AS',
                       use_unicode=True, 
                       charset="utf8")
cur = conn.cursor()

for i in id_names_list:
    print(i)
    p_id = i[0]
    p_name = i[1]
    p_source = i[2]
    
    '''updating ids of table person_doc with history-lab ids'''
    query = "SELECT id FROM persons5 WHERE person_name=%s AND source = 'cpdoc'"
    cur.execute(query, (p_name))
    #print(cur.fetchall())
    old_id = cur.fetchall()[0][0]
    query = "DELETE FROM person_doc5 WHERE person_id=%s" # só pode rodar isso 1 vez. Caso contrário, vai deletar ids corretas!
    cur.execute(query, (p_id))
    query = "UPDATE person_doc5 SET person_id =%s WHERE person_id=%s"
    cur.execute(query, (p_id, old_id))
        
    '''updating ids of table persons with history-lab ids'''
    query = "DELETE FROM persons5 WHERE id=%s"
    cur.execute(query, (p_id))
    query = "UPDATE persons5 SET id =%s WHERE person_name=%s AND source = 'cpdoc'"
    cur.execute(query, (p_id, p_name))

    '''fixing source'''
    query = "UPDATE persons5 SET source =%s WHERE person_name=%s AND source = 'cpdoc'"
    cur.execute(query, (p_source, p_name))
    
'''renaming source with badly chosen name'''
cur.execute("UPDATE persons5 SET source = 'frus and cpdoc' WHERE source = 'duplicate'")

········
[7162, 'mário soares', 'clinton and cpdoc']
['114246', 'vernon walters', 'frus and cpdoc']
['116279', 'frederick wills', 'frus and cpdoc']
['104230', 'raul fernandes', 'frus and cpdoc']
[2877, 'lincoln gordon', 'clinton and cpdoc']
[1183, 'rafael caldera', 'clinton and cpdoc']
['115232', 'juan maria bordaberry', 'frus and cpdoc']
[47, 'dean acheson', 'clinton and cpdoc']
['111914', 'hernan santa cruz', 'frus and cpdoc']
['101368', 'eugene black', 'frus and cpdoc']
['106300', 'samuel huntington', 'frus and cpdoc']
['109882', 'richard nixon', 'frus and cpdoc']
['111954', 'eisaku sato', 'frus and cpdoc']
['102333', 'nicolae ceausescu', 'frus and cpdoc']
[111487, 'nelson rockefeller', 'kissinger and cpdoc']
[5091, 'emílio garrastazu médici', 'clinton and cpdoc']
[6267, 'jânio quadros', 'clinton and cpdoc']
['115847', 'aparício mendez', 'frus and cpdoc']
['102566', 'winston churchill', 'frus and cpdoc']
['107024', 'edward kennedy', 'frus and cpdoc']
['104698', 'carlos garcia', 'fru

67

In [60]:
pass_mysql = getpass.getpass()
conn = pymysql.connect(host='localhost', 
                       user=sql_user,
                       passwd=pass_mysql,
                       db='CPDOC_AS',
                       use_unicode=True, 
                       charset="utf8")
cur = conn.cursor()

#cur.execute("DROP TABLE IF EXISTS persons6")
#cur.execute('''CREATE TABLE IF NOT EXISTS persons6
#           (id VARCHAR(128) PRIMARY KEY, person_name VARCHAR(128), source VARCHAR(128),
#           birth_year INT(4), death_year INT(4), description MEDIUMTEXT
#           DEFAULT NULL)
#           ENGINE=MyISAM DEFAULT CHARSET='utf8';''')

cur.execute("DROP TABLE IF EXISTS person_doc6")
cur.execute('''CREATE TABLE IF NOT EXISTS person_doc6
           (person_id VARCHAR(128), doc_id VARCHAR(31), person_count SMALLINT(5), date DATETIME,
           FOREIGN KEY (person_id) REFERENCES persons6(id)
           )
           ENGINE=MyISAM DEFAULT CHARSET='utf8';''')

········


/usr/local/lib/python3.5/dist-packages/pymysql/cursors.py:166: Warning: (1051, "Unknown table 'CPDOC_AS.person_doc6'")
  result = self._query(query)


0